In [9]:
from keras.models import load_model
from time import sleep
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
import time

face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
classifier = load_model(r'final_emotion_modelG1.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

def draw_progress_bar(img, emotion, value, x, y, width=200, height=20):
    cv2.rectangle(img, (x, y), (x + width, y + height), (255, 255, 255), 1)
    filled_width = int(width * value)
    cv2.rectangle(img, (x, y), (x + filled_width, y + height), (0, 255, 0), -1)
    
    text = f"{emotion}: {value:.2f}"
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    text_x = x + (width - text_width) // 2
    text_y = y + height - (height - text_height) // 2
    
    cv2.putText(img, text, (text_x - 1, text_y - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

cap = cv2.VideoCapture(0)

emotion_start_time = time.time()
last_dominant_emotion = 'Neutral'

while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation=cv2.INTER_AREA)
        
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            prediction = classifier.predict(roi)[0]
            label = emotion_labels[prediction.argmax()]
            label_position = (x, y)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            
            # Draw progress bars
            for i, (emotion, value) in enumerate(zip(emotion_labels, prediction)):
                draw_progress_bar(frame, emotion, value, 10, 50 + i * 25)
            
            # Update dominant emotion
            dominant_emotion = emotion_labels[prediction.argmax()]
            if dominant_emotion != last_dominant_emotion:
                emotion_start_time = time.time()
                last_dominant_emotion = dominant_emotion
            
            # Draw emotion indicator circle
            circle_x = 30
            circle_y = frame.shape[0] - 30
            circle_color = (255, 255, 255)  # Default white
            circle_radius = 10  # Initial size

            emotion_duration = time.time() - emotion_start_time
            if emotion_duration > 5:
                circle_radius = 15  # Scale up after 5 seconds
                if dominant_emotion == 'Happy':
                    circle_color = (0, 255, 0)  # Green
                elif dominant_emotion == 'Sad':
                    circle_color = (0, 165, 255)  # Orange
                elif dominant_emotion in ['Fear', 'Angry']:
                    circle_color = (0, 0, 255)  # Red
                elif dominant_emotion in ['Surprise', 'Disgust']:
                    circle_color = (0, 255, 255)  # Yellow

            if emotion_duration > 60 and dominant_emotion in ['Sad', 'Neutral']:
                circle_color = (255, 0, 0)  # Blue
                cv2.putText(frame, "Autism risk", (circle_x - 30, circle_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            cv2.circle(frame, (circle_x, circle_y), circle_radius, circle_color, -1)
        else:
            cv2.putText(frame, 'No Faces', (30,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    
    cv2.imshow('Emotion Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cvzone'

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import keras
print("Keras version:", keras.__version__)

import cv2
print("OpenCV version:", cv2.__version__)

TensorFlow version: 2.17.0
Keras version: 3.5.0
OpenCV version: 4.10.0


In [4]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.17.0
Num GPUs Available:  0
[]
